In [7]:
# Parte 1: Traduzione e salvataggio delle traduzioni
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch

# Carica i modelli di traduzione e i tokenizer
model_names = [
    "Helsinki-NLP/opus-mt-en-it",
    "Gopal1853/Gopal-finetuned-custom-en-to-it",
    "Anhptp/opus-mt-en-it-BDS-G1",
    "callmyname/marian-finetuned-kde4-en-to-it"
]

models = {}
tokenizers = {}
for model_name in model_names:
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    models[model_name] = model
    tokenizers[model_name] = tokenizer

# Carica il corpus
file_input_csv = 'europarl_unito_500righe.csv'
df = pd.read_csv(file_input_csv, delimiter=';')

translations_dict = {model_name: [] for model_name in model_names}

for model_name in model_names:
    model = models[model_name]
    tokenizer = tokenizers[model_name]

    english_texts = df['Inglese'].tolist()

    translations = []
    for text in english_texts:
        if isinstance(text, str):
            inputs = tokenizer(text, return_tensors="pt", padding=True)
            outputs = model.generate(**inputs)
            translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
            translations.append(translation)
        else:
            print(f"Errore: Il testo non è di tipo str, ma {type(text)}")
            translations.append("")

    translations_dict[model_name] = translations

# Salva le traduzioni in un file CSV
file_translations_csv = 'europarl_translations.csv'
df_output = pd.DataFrame()
df_output['Italiano'] = df['Italiano']
df_output['Inglese'] = df['Inglese']

for model_name, translations in translations_dict.items():
    df_output[f'{model_name} (Traduzione)'] = translations

df_output.to_csv(file_translations_csv, index=False, sep=';')
print(f"File CSV con traduzioni creato: {file_translations_csv}")

tokenizer_config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/789k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/814k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/341M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

Errore: Il testo non è di tipo str, ma <class 'float'>
Errore: Il testo non è di tipo str, ma <class 'float'>
Errore: Il testo non è di tipo str, ma <class 'float'>
Errore: Il testo non è di tipo str, ma <class 'float'>
File CSV con traduzioni creato: europarl_translations.csv


In [8]:
# Parte 2: Calcolo dei punteggi BLEU
from nltk.translate.bleu_score import corpus_bleu

def calculate_bleu(df, translations_dict):
    scores = {}

    # Converti la colonna "Italiano" in stringhe e sostituisci eventuali NaN
    original_texts = df['Italiano'].fillna("").astype(str).tolist()

    for model_name, translations in translations_dict.items():
        bleu_score = corpus_bleu(
            [[ref.split()] for ref in original_texts if isinstance(ref, str)],
            [hyp.split() for hyp in translations if isinstance(hyp, str)]
        )
        scores[model_name] = bleu_score

    return scores

# Calcola i punteggi BLEU
bleu_scores = calculate_bleu(df, translations_dict)

# Stampa i punteggi BLEU per tutti i modelli
for model_name, score in bleu_scores.items():
    print(f"Modello: {model_name}, Punteggio BLEU: {score}")

# Trova il modello migliore
best_model = max(bleu_scores, key=bleu_scores.get)
best_model_score = bleu_scores[best_model]

print(f"\nIl miglior modello è: {best_model} con punteggio BLEU: {best_model_score}")

# Salva i punteggi BLEU nel file CSV
for model_name, score in bleu_scores.items():
    df_output[f'{model_name} (BLEU: {score:.4f})'] = translations_dict[model_name]

file_output_csv = 'europarl_bleu.csv'
df_output.to_csv(file_output_csv, index=False, sep=';')
print(f"File CSV con punteggi BLEU creato: {file_output_csv}")

Modello: Helsinki-NLP/opus-mt-en-it, Punteggio BLEU: 0.23570952086490674
Modello: Gopal1853/Gopal-finetuned-custom-en-to-it, Punteggio BLEU: 0.21251042633583053
Modello: Anhptp/opus-mt-en-it-BDS-G1, Punteggio BLEU: 0.22235800555614482
Modello: callmyname/marian-finetuned-kde4-en-to-it, Punteggio BLEU: 0.14978396226125565

Il miglior modello è: Helsinki-NLP/opus-mt-en-it con punteggio BLEU: 0.23570952086490674
File CSV con punteggi BLEU creato: europarl_bleu.csv


In [1]:
import pandas as pd

def detect_delimiter(filepath):
    # Legge il file CSV provando con diversi delimitatori comuni
    sample_size = 1000  # Numero di righe da leggere per determinare il delimitatore
    sample_lines = pd.read_csv(filepath, nrows=sample_size, sep=None, engine='python')

    # Ottiene i possibili delimitatori
    delimiters = [',', ';', '\t', '|']

    for delim in delimiters:
        try:
            # Prova a leggere con il delimitatore attuale
            sample_lines = pd.read_csv(filepath, delimiter=delim, nrows=sample_size)
            print(f"Delimiter trovato: '{delim}'")
            return delim
        except pd.errors.ParserError:
            continue

    print("Delimitatore non riconosciuto.")
    return None

# Esempio di utilizzo
file_path = 'merged_translcsv'
detected_delimiter = detect_delimiter(file_path)
if detected_delimiter:
    print(f"Il delimitatore trovato è: '{detected_delimiter}'")
else:
    print("Non è stato possibile identificare il delimitatore.")


Delimiter trovato: ','
Il delimitatore trovato è: ','
